# Random Forest

In [1]:
cd "E:\pycham\项目\pythonProject\coda\8分类"

E:\pycham\项目\pythonProject\coda\8分类


E:\pycham\项目\pythonProject\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import pandas as pd

In [3]:
results = pd.DataFrame(columns=['Models', 'AUC',
                                'Avg. Sensitivity', 'Avg. Specificity',
                                'PPV (Recall)', 'NPV (Precision)',
                                'Accuracy', 'F1 Score'])


In [6]:
# from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
import numpy as np
def report_metrics(y_pred_prob, y_test, model_name):
    y_pred = y_pred_prob.argmax(axis=1)
    # Calculating AUC
    auc = roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average=None)
    print(f"AUC: {auc}")
    # Calculating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    # Calculating sensitivity and specificity for each class
    num_classes = 8
    sensitivity = np.zeros(num_classes)
    specificity = np.zeros(num_classes)
    for i in range(num_classes):
        true_positive = conf_matrix[i, i]
        false_negative = np.sum(conf_matrix[i, :]) - true_positive
        false_positive = np.sum(conf_matrix[:, i]) - true_positive
        true_negative = np.sum(conf_matrix) - true_positive - false_positive - false_negative
        sensitivity[i] = true_positive / (true_positive + false_negative)
        specificity[i] = true_negative / (true_negative + false_positive)

    # Calculating precision, recall, and F1 score
    precision = precision_score(y_test, y_pred, average=None)
    recall = recall_score(y_test, y_pred, average=None)
    f1 = f1_score(y_test, y_pred, average=None)

    accuracy = accuracy_score(y_test, y_pred)
    # Reporting the evaluation metrics
    for i in range(num_classes):
        print(f"Class {i+1} - Sensitivity: {sensitivity[i]}, Specificity: {specificity[i]}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Averaged sensitivity: {np.mean(sensitivity)}")
    print(f"Averaged specificity: {np.mean(specificity)}")
    newd = {'Models': model_name,
            'AUC': auc,
            'Avg. Sensitivity': np.mean(sensitivity),
            'Avg. Specificity': np.mean(specificity),
            'PPV (Recall)': np.mean(recall),
            'NPV (Precision)': np.mean(precision),
            'Accuracy': np.mean(accuracy),
            'F1 Score': f1
            }
    return newd


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold

# Step 1: Split data into train and test set
data = pd.read_csv('/content/drive/MyDrive/parttime/peak_table.csv')

from util import get_labels
y = get_labels(data)
print(y.shape)

X = data.drop(columns=['group', 'label'], axis=1)

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train a random forest model
    rf_model = RandomForestClassifier()
    rf_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_prob = rf_model.predict_proba(x_fold_val)
    report_metrics(y_pred_prob, y_fold_val, 'rf')

[0 0 0 0 0]
(1072,)
**********fold--0**********

AUC: [0.98945946 0.99267705 0.92297297 1.         0.99942704 0.99828829
 0.99168077 0.99828829]
Class 1 - Sensitivity: 0.9333333333333333, Specificity: 0.9837837837837838
Class 2 - Sensitivity: 0.896551724137931, Specificity: 0.9623655913978495
Class 3 - Sensitivity: 0.6666666666666666, Specificity: 0.9567567567567568
Class 4 - Sensitivity: 0.95, Specificity: 1.0
Class 5 - Sensitivity: 0.9642857142857143, Specificity: 1.0
Class 6 - Sensitivity: 1.0, Specificity: 0.9621621621621622
Class 7 - Sensitivity: 0.5, Specificity: 1.0
Class 8 - Sensitivity: 0.9666666666666667, Specificity: 0.9891891891891892
Precision: [0.90322581 0.78787879 0.71428571 1.         1.         0.81081081
 1.         0.93548387]
Recall: [0.93333333 0.89655172 0.66666667 0.95       0.96428571 1.
 0.5        0.96666667]
Accuracy: 0.8744186046511628
F1 Score: [0.91803279 0.83870968 0.68965517 0.97435897 0.98181818 0.89552239
 0.66666667 0.95081967]
Averaged sensitivity: 

In [ ]:
# results = results.append(report_metrics(conf_matrix, auc, 'RF'), ignore_index=True)

Class 1 - Sensitivity: 0.972972972972973, Specificity: 1.0
Class 2 - Sensitivity: 0.9696969696969697, Specificity: 1.0
Class 3 - Sensitivity: 0.9696969696969697, Specificity: 0.9945054945054945
Class 4 - Sensitivity: 0.9473684210526315, Specificity: 0.9948979591836735
Class 5 - Sensitivity: 1.0, Specificity: 0.9895833333333334
Class 6 - Sensitivity: 1.0, Specificity: 0.9946808510638298
Class 7 - Sensitivity: 0.9375, Specificity: 1.0
Class 8 - Sensitivity: 0.9629629629629629, Specificity: 0.9946808510638298
Precision: 0.9705392584997847
Recall: 0.9700247870478134
Accuracy: 0.9720930232558139
F1 Score: 0.9698548198532936
Averaged sensitivity: 0.9700247870478134
Averaged specificity: 0.9960435611437701


<ipython-input-14-483786ebac56>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(conf_matrix, auc, 'RF'), ignore_index=True)


In [ ]:
# results

,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855


# GBM

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train a GBM model
    gbm_model = GradientBoostingClassifier()
    gbm_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = gbm_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'gbm')

**********fold--0**********

AUC: [0.98882883 0.98906192 0.89981982 1.         1.         0.99423423
 0.98392555 0.98792793]
Class 1 - Sensitivity: 0.9333333333333333, Specificity: 0.9945945945945946
Class 2 - Sensitivity: 0.9310344827586207, Specificity: 0.9516129032258065
Class 3 - Sensitivity: 0.7, Specificity: 0.972972972972973
Class 4 - Sensitivity: 1.0, Specificity: 1.0
Class 5 - Sensitivity: 1.0, Specificity: 0.983957219251337
Class 6 - Sensitivity: 0.9333333333333333, Specificity: 0.972972972972973
Class 7 - Sensitivity: 0.5555555555555556, Specificity: 0.9949238578680203
Class 8 - Sensitivity: 0.8, Specificity: 0.972972972972973
Precision: [0.96551724 0.75       0.80769231 1.         0.90322581 0.84848485
 0.90909091 0.82758621]
Recall: [0.93333333 0.93103448 0.7        1.         1.         0.93333333
 0.55555556 0.8       ]
Accuracy: 0.8651162790697674
F1 Score: [0.94915254 0.83076923 0.75       1.         0.94915254 0.88888889
 0.68965517 0.81355932]
Averaged sensitivity: 0

In [ ]:
# y_true_binary = label_binarize(y_test, classes=np.arange(8))


In [ ]:
# y_pred = y_pred_proba.argmax(axis=1)

In [ ]:
# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')

# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)

In [ ]:
# print(auc)

0.9998501831457607


In [ ]:
# results = results.append(report_metrics(cm, auc, 'GBM'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.972972972972973, Specificity: 1.0
Class 2 - Sensitivity: 0.9696969696969697, Specificity: 1.0
Class 3 - Sensitivity: 1.0, Specificity: 0.9945054945054945
Class 4 - Sensitivity: 0.9473684210526315, Specificity: 0.9948979591836735
Class 5 - Sensitivity: 1.0, Specificity: 0.9947916666666666
Class 6 - Sensitivity: 1.0, Specificity: 1.0
Class 7 - Sensitivity: 1.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 1.0
Precision: 0.9845362487100103
Recall: 0.9862547954653218
Accuracy: 0.986046511627907
F1 Score: 0.9852604008315026
Averaged sensitivity: 0.9862547954653218
Averaged specificity: 0.9980243900444794


<ipython-input-22-8f8f97c7caf7>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'GBM'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260


# XGBoost

In [17]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix


kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train an XGBoost model
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = xgb_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'xgboost')
# y_pred = y_pred_proba.argmax(axis=1)


# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.98810811 0.97979236 0.93765766 1.         0.99980901 0.99927928
 0.99266779 0.99711712]
Class 1 - Sensitivity: 0.9, Specificity: 0.972972972972973
Class 2 - Sensitivity: 0.9655172413793104, Specificity: 0.956989247311828
Class 3 - Sensitivity: 0.6666666666666666, Specificity: 0.9837837837837838
Class 4 - Sensitivity: 0.85, Specificity: 1.0
Class 5 - Sensitivity: 1.0, Specificity: 0.9732620320855615
Class 6 - Sensitivity: 0.9666666666666667, Specificity: 0.9891891891891892
Class 7 - Sensitivity: 0.7777777777777778, Specificity: 1.0
Class 8 - Sensitivity: 0.9666666666666667, Specificity: 1.0
Precision: [0.84375    0.77777778 0.86956522 1.         0.84848485 0.93548387
 1.         1.        ]
Recall: [0.9        0.96551724 0.66666667 0.85       1.         0.96666667
 0.77777778 0.96666667]
Accuracy: 0.8930232558139535
F1 Score: [0.87096774 0.86153846 0.75471698 0.91891892 0.91803279 0.95081967
 0.875      0.98305085]
Averaged sensitivity: 0.88666187739

In [ ]:
# results = results.append(report_metrics(cm, auc, 'XGBoost'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.972972972972973, Specificity: 1.0
Class 2 - Sensitivity: 1.0, Specificity: 0.9945054945054945
Class 3 - Sensitivity: 0.9393939393939394, Specificity: 1.0
Class 4 - Sensitivity: 1.0, Specificity: 0.9948979591836735
Class 5 - Sensitivity: 1.0, Specificity: 1.0
Class 6 - Sensitivity: 1.0, Specificity: 1.0
Class 7 - Sensitivity: 1.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 0.9946808510638298
Precision: 0.985609243697479
Recall: 0.989045864045864
Accuracy: 0.986046511627907
F1 Score: 0.9870378941132303
Averaged sensitivity: 0.989045864045864
Averaged specificity: 0.9980105380941248


<ipython-input-24-50325613c222>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'XGBoost'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038


# AdaBoost

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix


kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train an AdaBoost model
    adaboost_model = AdaBoostClassifier()
    adaboost_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = adaboost_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'adaboost')
# y_pred = y_pred_proba.argmax(axis=1)

# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.6172973  0.41889136 0.82918919 0.54615385 0.89839572 0.69963964
 0.82642414 0.86882883]
Class 1 - Sensitivity: 0.8, Specificity: 0.7135135135135136
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 0.8333333333333334, Specificity: 0.8
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.4444444444444444, Specificity: 0.9593908629441624
Class 8 - Sensitivity: 0.8666666666666667, Specificity: 0.8162162162162162
Precision: [0.31168831 0.         0.40322581 0.         0.         0.
 0.5        0.43333333]
Recall: [0.8        0.         0.83333333 0.         0.         0.
 0.44444444 0.86666667]
Accuracy: 0.386046511627907
F1 Score: [0.44859813 0.         0.54347826 0.         0.         0.
 0.47058824 0.57777778]
Averaged sensitivity: 0.3680555555555556
Averaged specificity: 0.9111400740842366
**********fold--1**********


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC: [0.48027027 0.49888765 0.84882883 0.42051282 0.83489305 0.73567568
 0.78257191 0.88558559]
Class 1 - Sensitivity: 0.9, Specificity: 0.7567567567567568
Class 2 - Sensitivity: 0.0, Specificity: 0.9838709677419355
Class 3 - Sensitivity: 0.23333333333333334, Specificity: 0.8432432432432433
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.3, Specificity: 0.7783783783783784
Class 7 - Sensitivity: 0.0, Specificity: 0.9898477157360406
Class 8 - Sensitivity: 0.8333333333333334, Specificity: 0.8540540540540541
Precision: [0.375      0.         0.19444444 0.         0.         0.18
 0.         0.48076923]
Recall: [0.9        0.         0.23333333 0.         0.         0.3
 0.         0.83333333]
Accuracy: 0.31627906976744186
F1 Score: [0.52941176 0.         0.21212121 0.         0.         0.225
 0.         0.6097561 ]
Averaged sensitivity: 0.2833333333333333
Averaged specificity: 0.9007688894888011
**********fold--2**********

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC: [0.88922101 0.69338304 0.74094203 0.96649485 0.80885177 0.83699907
 0.96740363 0.89266304]
Class 1 - Sensitivity: 0.3333333333333333, Specificity: 0.9456521739130435
Class 2 - Sensitivity: 0.5172413793103449, Specificity: 0.8918918918918919
Class 3 - Sensitivity: 0.23333333333333334, Specificity: 0.875
Class 4 - Sensitivity: 0.75, Specificity: 0.9639175257731959
Class 5 - Sensitivity: 0.6428571428571429, Specificity: 0.9354838709677419
Class 6 - Sensitivity: 0.4827586206896552, Specificity: 0.8216216216216217
Class 7 - Sensitivity: 0.1111111111111111, Specificity: 0.9795918367346939
Class 8 - Sensitivity: 0.43333333333333335, Specificity: 0.9402173913043478
Precision: [0.5        0.42857143 0.23333333 0.68181818 0.6        0.29787234
 0.33333333 0.54166667]
Recall: [0.33333333 0.51724138 0.23333333 0.75       0.64285714 0.48275862
 0.11111111 0.43333333]
Accuracy: 0.4392523364485981
F1 Score: [0.4        0.46875    0.23333333 0.71428571 0.62068966 0.36842105
 0.16666667 0.48148148

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC: [0.63414855 0.55357143 0.52173913 0.91469072 0.60661696 0.73243243
 0.71938776 0.80208333]
Class 1 - Sensitivity: 0.1, Specificity: 0.9945652173913043
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 1.0, Specificity: 0.46195652173913043
Class 4 - Sensitivity: 0.7, Specificity: 0.9278350515463918
Class 5 - Sensitivity: 0.20689655172413793, Specificity: 0.8216216216216217
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.0, Specificity: 1.0
Class 8 - Sensitivity: 0.0, Specificity: 0.9239130434782609
Precision: [0.75       0.         0.23255814 0.5        0.15384615 0.
 0.         0.        ]
Recall: [0.1        0.         1.         0.7        0.20689655 0.
 0.         0.        ]
Accuracy: 0.24766355140186916
F1 Score: [0.17647059 0.         0.37735849 0.58333333 0.17647059 0.
 0.         0.        ]
Averaged sensitivity: 0.25086206896551727
Averaged specificity: 0.8912364319720887


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# results = results.append(report_metrics(cm, auc, 'AdaBoost'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.1891891891891892, Specificity: 0.9269662921348315
Class 2 - Sensitivity: 0.0, Specificity: 0.9725274725274725
Class 3 - Sensitivity: 0.7575757575757576, Specificity: 0.6923076923076923
Class 4 - Sensitivity: 0.8421052631578947, Specificity: 0.7857142857142857
Class 5 - Sensitivity: 0.9130434782608695, Specificity: 0.9791666666666666
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.1875, Specificity: 0.9798994974874372
Class 8 - Sensitivity: 0.5555555555555556, Specificity: 0.9787234042553191
Precision: 0.3740686446320142
Recall: 0.4306211554674083
Accuracy: 0.4046511627906977
F1 Score: 0.3609798025201343
Averaged sensitivity: 0.4306211554674083
Averaged specificity: 0.9144131638867131


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-26-3a0a81aaa357>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'AdaBoost'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980


In [ ]:
# y_pred

# DT

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train a Decision Tree model
    decision_tree_model = DecisionTreeClassifier()
    decision_tree_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = decision_tree_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'dt')
# y_pred = y_pred_proba.argmax(axis=1)


# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')

# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.75945946 0.80608083 0.72657658 0.7224359  0.90718105 0.82837838
 0.68429216 0.8509009 ]
Class 1 - Sensitivity: 0.6, Specificity: 0.918918918918919
Class 2 - Sensitivity: 0.6551724137931034, Specificity: 0.956989247311828
Class 3 - Sensitivity: 0.5666666666666667, Specificity: 0.8864864864864865
Class 4 - Sensitivity: 0.45, Specificity: 0.9948717948717949
Class 5 - Sensitivity: 0.8571428571428571, Specificity: 0.9572192513368984
Class 6 - Sensitivity: 0.7, Specificity: 0.9567567567567568
Class 7 - Sensitivity: 0.3888888888888889, Specificity: 0.9796954314720813
Class 8 - Sensitivity: 0.7666666666666667, Specificity: 0.9351351351351351
Precision: [0.54545455 0.7037037  0.44736842 0.9        0.75       0.72413793
 0.63636364 0.65714286]
Recall: [0.6        0.65517241 0.56666667 0.45       0.85714286 0.7
 0.38888889 0.76666667]
Accuracy: 0.641860465116279
F1 Score: [0.57142857 0.67857143 0.5        0.6        0.8        0.71186441
 0.48275862 0.70769231

In [ ]:
# results = results.append(report_metrics(cm, auc, 'DT'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.7837837837837838, Specificity: 0.9887640449438202
Class 2 - Sensitivity: 0.9090909090909091, Specificity: 0.9945054945054945
Class 3 - Sensitivity: 0.8787878787878788, Specificity: 0.978021978021978
Class 4 - Sensitivity: 1.0, Specificity: 0.9795918367346939
Class 5 - Sensitivity: 1.0, Specificity: 0.984375
Class 6 - Sensitivity: 0.9259259259259259, Specificity: 0.9787234042553191
Class 7 - Sensitivity: 0.8125, Specificity: 0.9949748743718593
Class 8 - Sensitivity: 0.9259259259259259, Specificity: 0.9840425531914894
Precision: 0.8970266954153036
Recall: 0.904501802939303
Accuracy: 0.8976744186046511
F1 Score: 0.8976726486048965
Averaged sensitivity: 0.904501802939303
Averaged specificity: 0.9853748982530818


<ipython-input-29-6545e90d28e3>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'DT'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980
4,DT,0.944938,0.904502,0.985375,0.904502,0.897027,0.897674,0.897673


# ET

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train an Extra Trees model
    extra_trees_model = ExtraTreesClassifier()
    extra_trees_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = extra_trees_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'ET')
# y_pred = y_pred_proba.argmax(axis=1)

# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.99153153 0.9873934  0.92018018 1.         0.99971352 0.99927928
 0.99069374 0.99963964]
Class 1 - Sensitivity: 0.9333333333333333, Specificity: 0.972972972972973
Class 2 - Sensitivity: 0.8620689655172413, Specificity: 0.9623655913978495
Class 3 - Sensitivity: 0.6666666666666666, Specificity: 0.9351351351351351
Class 4 - Sensitivity: 0.85, Specificity: 1.0
Class 5 - Sensitivity: 0.9642857142857143, Specificity: 1.0
Class 6 - Sensitivity: 1.0, Specificity: 0.9621621621621622
Class 7 - Sensitivity: 0.3888888888888889, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 1.0
Precision: [0.84848485 0.78125    0.625      1.         1.         0.81081081
 1.         1.        ]
Recall: [0.93333333 0.86206897 0.66666667 0.85       0.96428571 1.
 0.38888889 1.        ]
Accuracy: 0.8558139534883721
F1 Score: [0.88888889 0.81967213 0.64516129 0.91891892 0.98181818 0.89552239
 0.56       1.        ]
Averaged sensitivity: 0.8331554460864805
Averaged specifi

In [ ]:
# results = results.drop(results.index[-1])

In [ ]:
# results = results.append(report_metrics(cm, auc, 'ET'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.972972972972973, Specificity: 1.0
Class 2 - Sensitivity: 1.0, Specificity: 1.0
Class 3 - Sensitivity: 0.9696969696969697, Specificity: 1.0
Class 4 - Sensitivity: 0.9473684210526315, Specificity: 0.9948979591836735
Class 5 - Sensitivity: 1.0, Specificity: 0.9947916666666666
Class 6 - Sensitivity: 1.0, Specificity: 1.0
Class 7 - Sensitivity: 1.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 0.9946808510638298
Precision: 0.9837484335839599
Recall: 0.9862547954653218
Accuracy: 0.986046511627907
F1 Score: 0.9848533452005663
Averaged sensitivity: 0.9862547954653218
Averaged specificity: 0.9980463096142713


<ipython-input-32-27839b7e9755>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'ET'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980
4,DT,0.944938,0.904502,0.985375,0.904502,0.897027,0.897674,0.897673
5,ET,0.999639,0.986255,0.998046,0.986255,0.983748,0.986047,0.984853


# SVC

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train an SVM model
    svm_model = SVC(probability=True)
    svm_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = svm_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'SVC')
# y_pred = y_pred_proba.argmax(axis=1)

# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.99765766 0.94215795 0.91531532 1.         1.         0.99693694
 0.99830795 1.        ]
Class 1 - Sensitivity: 0.9, Specificity: 0.9837837837837838
Class 2 - Sensitivity: 0.7586206896551724, Specificity: 0.8924731182795699
Class 3 - Sensitivity: 0.43333333333333335, Specificity: 0.972972972972973
Class 4 - Sensitivity: 1.0, Specificity: 1.0
Class 5 - Sensitivity: 1.0, Specificity: 1.0
Class 6 - Sensitivity: 1.0, Specificity: 0.9783783783783784
Class 7 - Sensitivity: 0.7777777777777778, Specificity: 1.0
Class 8 - Sensitivity: 0.9666666666666667, Specificity: 1.0
Precision: [0.9        0.52380952 0.72222222 1.         1.         0.88235294
 1.         1.        ]
Recall: [0.9        0.75862069 0.43333333 1.         1.         1.
 0.77777778 0.96666667]
Accuracy: 0.8511627906976744
F1 Score: [0.9        0.61971831 0.54166667 1.         1.         0.9375
 0.875      0.98305085]
Averaged sensitivity: 0.8545498084291188
Averaged specificity: 0.97845103167

In [ ]:
# results = results.append(report_metrics(cm, auc, 'SVC'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.9459459459459459, Specificity: 0.9887640449438202
Class 2 - Sensitivity: 0.9393939393939394, Specificity: 0.9835164835164835
Class 3 - Sensitivity: 0.9090909090909091, Specificity: 0.989010989010989
Class 4 - Sensitivity: 0.9473684210526315, Specificity: 0.9948979591836735
Class 5 - Sensitivity: 1.0, Specificity: 0.9947916666666666
Class 6 - Sensitivity: 0.9629629629629629, Specificity: 0.9893617021276596
Class 7 - Sensitivity: 0.875, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 1.0
Precision: 0.9536854793482115
Recall: 0.9474702723057986
Accuracy: 0.9488372093023256
F1 Score: 0.949909463430882
Averaged sensitivity: 0.9474702723057986
Averaged specificity: 0.9925428556811615


<ipython-input-34-44615b555774>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'SVC'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980
4,DT,0.944938,0.904502,0.985375,0.904502,0.897027,0.897674,0.897673
5,ET,0.999639,0.986255,0.998046,0.986255,0.983748,0.986047,0.984853
6,SVC,0.997607,0.947470,0.992543,0.947470,0.953685,0.948837,0.949909


# LR

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix



kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train a Logistic Regression model
    logistic_regression_model = LogisticRegression()
    logistic_regression_model.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = logistic_regression_model.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'LR')
# y_pred = y_pred_proba.argmax(axis=1)

# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.75531532 0.6575825  0.84864865 0.97666667 0.96638655 0.82036036
 0.86520023 0.78396396]
Class 1 - Sensitivity: 0.6, Specificity: 0.8054054054054054
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 0.9, Specificity: 0.6432432432432432
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.0, Specificity: 1.0
Class 8 - Sensitivity: 0.9333333333333333, Specificity: 0.7837837837837838
Precision: [0.33333333 0.         0.29032258 0.         0.         0.
 0.         0.41176471]
Recall: [0.6        0.         0.9        0.         0.         0.
 0.         0.93333333]
Accuracy: 0.3395348837209302
F1 Score: [0.42857143 0.         0.43902439 0.         0.         0.
 0.         0.57142857]
Averaged sensitivity: 0.3041666666666667
Averaged specificity: 0.904054054054054
**********fold--1**********


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC: [0.74324324 0.78068224 0.85351351 0.91897436 0.94499618 0.83153153
 0.8355894  0.92900901]
Class 1 - Sensitivity: 0.9, Specificity: 0.7675675675675676
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 1.0, Specificity: 0.6324324324324324
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 0.9081081081081082
Precision: [0.38571429 0.         0.30612245 0.         0.         0.
 0.         0.63829787]
Recall: [0.9 0.  1.  0.  0.  0.  0.  1. ]
Accuracy: 0.4046511627906977
F1 Score: [0.54       0.         0.46875    0.         0.         0.
 0.         0.77922078]
Averaged sensitivity: 0.3625
Averaged specificity: 0.9135135135135135
**********fold--2**********

AUC: [0.6490942  0.677726   0.79565217 0.90180412 0.87423195 0.83988816
 0.96031746 0.925     ]
Class 1 - Sensitivity: 0.6, Specificity: 0.706

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AUC: [0.82264493 0.90214352 0.83949275 0.94536082 0.8312212  0.79571295
 0.90391156 0.94963768]
Class 1 - Sensitivity: 0.7666666666666667, Specificity: 0.8532608695652174
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 0.9333333333333333, Specificity: 0.532608695652174
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.0, Specificity: 1.0
Class 8 - Sensitivity: 0.9333333333333333, Specificity: 0.8804347826086957
Precision: [0.46       0.         0.24561404 0.         0.         0.
 0.         0.56      ]
Recall: [0.76666667 0.         0.93333333 0.         0.         0.
 0.         0.93333333]
Accuracy: 0.3691588785046729
F1 Score: [0.575      0.         0.38888889 0.         0.         0.
 0.         0.7       ]
Averaged sensitivity: 0.3291666666666667
Averaged specificity: 0.9082880434782609
**********fold--4**********

AUC: [0.89112319 0.72619048 0.8175724

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# results = results.append(report_metrics(cm, auc, 'LR'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.0, Specificity: 1.0
Class 2 - Sensitivity: 0.0, Specificity: 1.0
Class 3 - Sensitivity: 0.0, Specificity: 1.0
Class 4 - Sensitivity: 0.0, Specificity: 1.0
Class 5 - Sensitivity: 0.0, Specificity: 1.0
Class 6 - Sensitivity: 0.0, Specificity: 1.0
Class 7 - Sensitivity: 0.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 0.0
Precision: 0.015697674418604653
Recall: 0.125
Accuracy: 0.12558139534883722
F1 Score: 0.02789256198347108
Averaged sensitivity: 0.125
Averaged specificity: 0.875


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-36-cfa41479c952>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'LR'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980
4,DT,0.944938,0.904502,0.985375,0.904502,0.897027,0.897674,0.897673
5,ET,0.999639,0.986255,0.998046,0.986255,0.983748,0.986047,0.984853
6,SVC,0.997607,0.947470,0.992543,0.947470,0.953685,0.948837,0.949909
7,LR,0.857569,0.125000,0.875000,0.125000,0.015698,0.125581,0.027893


In [ ]:
# y_pred_proba[:5]

In [ ]:
# logistic_regression_model.classes_

# KNN

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, confusion_matrix

kf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(kf.split(X, y)):
    x_fold_train, x_fold_val, y_fold_train, y_fold_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Adjust test_size as needed
    print(f'**********fold--{i}**********')
    print()
    # Step 2: Train a KNN model
    knn = KNeighborsClassifier(n_neighbors=5)  # Adjust n_neighbors as needed
    knn.fit(x_fold_train, y_fold_train)

    # Step 3: Predict and calculate metrics
    y_pred_proba = knn.predict_proba(x_fold_val)
    report_metrics(y_pred_proba, y_fold_val, 'KNN')
# y_pred = knn.predict(X_test)

# Calculate AUC
# auc = roc_auc_score(y_true_binary, y_pred_proba, multi_class='ovr')
# print(auc)
# Calculate sensitivity, specificity, ppv, npv, accuracy, and F1-score
# cm = confusion_matrix(y_test, y_pred)
# report_metrics(cm)

**********fold--0**********

AUC: [0.96288288 0.95939933 0.77873874 1.         0.98080596 0.9772973
 0.90905245 1.        ]
Class 1 - Sensitivity: 0.9, Specificity: 0.9783783783783784
Class 2 - Sensitivity: 0.7586206896551724, Specificity: 0.9247311827956989
Class 3 - Sensitivity: 0.26666666666666666, Specificity: 0.9567567567567568
Class 4 - Sensitivity: 1.0, Specificity: 1.0
Class 5 - Sensitivity: 0.9642857142857143, Specificity: 1.0
Class 6 - Sensitivity: 0.9666666666666667, Specificity: 0.8702702702702703
Class 7 - Sensitivity: 0.16666666666666666, Specificity: 1.0
Class 8 - Sensitivity: 0.9666666666666667, Specificity: 1.0
Precision: [0.87096774 0.61111111 0.5        1.         1.         0.54716981
 1.         1.        ]
Recall: [0.9        0.75862069 0.26666667 1.         0.96428571 0.96666667
 0.16666667 0.96666667]
Accuracy: 0.7674418604651163
F1 Score: [0.8852459  0.67692308 0.34782609 1.         0.98181818 0.69879518
 0.28571429 0.98305085]
Averaged sensitivity: 0.748696633

In [ ]:
# results = results.append(report_metrics(cm, auc, 'KNN'), ignore_index=True)
# results

Class 1 - Sensitivity: 0.8648648648648649, Specificity: 1.0
Class 2 - Sensitivity: 1.0, Specificity: 0.967032967032967
Class 3 - Sensitivity: 0.9090909090909091, Specificity: 0.989010989010989
Class 4 - Sensitivity: 1.0, Specificity: 0.9846938775510204
Class 5 - Sensitivity: 0.8260869565217391, Specificity: 1.0
Class 6 - Sensitivity: 0.9629629629629629, Specificity: 0.9946808510638298
Class 7 - Sensitivity: 1.0, Specificity: 1.0
Class 8 - Sensitivity: 1.0, Specificity: 0.9946808510638298
Precision: 0.9468173608798609
Recall: 0.9453757116800595
Accuracy: 0.9395348837209302
F1 Score: 0.9429565174329224
Averaged sensitivity: 0.9453757116800595
Averaged specificity: 0.9912624419653295


<ipython-input-40-7804d4b49773>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(report_metrics(cm, auc, 'KNN'), ignore_index=True)


,Models,AUC,Avg. Sensitivity,Avg. Specificity,PPV (Recall),NPV (Precision),Accuracy,F1 Score
0,RF,0.999595,0.970025,0.996044,0.970025,0.970539,0.972093,0.969855
1,GBM,0.999850,0.986255,0.998024,0.986255,0.984536,0.986047,0.985260
2,XGBoost,0.999672,0.989046,0.998011,0.989046,0.985609,0.986047,0.987038
3,AdaBoost,0.794536,0.430621,0.914413,0.430621,0.374069,0.404651,0.360980
4,DT,0.944938,0.904502,0.985375,0.904502,0.897027,0.897674,0.897673
5,ET,0.999639,0.986255,0.998046,0.986255,0.983748,0.986047,0.984853
6,SVC,0.997607,0.947470,0.992543,0.947470,0.953685,0.948837,0.949909
7,LR,0.857569,0.125000,0.875000,0.125000,0.015698,0.125581,0.027893
8,KNN,0.998678,0.945376,0.991262,0.945376,0.946817,0.939535,0.942957


In [ ]:
# results.to_csv('allmodels.csv')